## VAERSData Cleanup

In [15]:
# Add the Pandas dependency.
import pandas as pd

In [16]:
# Files to load
data_csv = "../Resources/Data/2021VAERSData.csv"
vax_csv = "../Resources/Data/2021VAERSVAX.csv"
symptoms_csv = "../Resources/Data/2021VAERSSYMPTOMS.csv"

In [17]:
# VAERSData Dataframe
vaers_data = pd.read_csv(data_csv, index_col=None, header=0, encoding='latin1', low_memory=False)
vaers_data.head()

,version https://git-lfs.github.com/spec/v1
0,oid sha256:228e400f99311a8342aab5e20c873ef4363...
1,size 255665141


In [18]:
# converting NaN to 0 in vaers data
vaers_data = vaers_data.fillna(0)

In [19]:
# checking if it worked. it did.
vaers_data.isnull().sum()

version https://git-lfs.github.com/spec/v1    0
dtype: int64

In [20]:
# Set VAERS_ID as index
vaers_data_df = vaers_data.set_index(["VAERS_ID"])
vaers_data_df.head()

KeyError: "None of ['VAERS_ID'] are in the columns"

In [21]:
# Check vaers_data_df columns
vaers_data_df.columns

NameError: name 'vaers_data_df' is not defined

In [22]:
# Drop  Columns 
clean_vaers_data_df = vaers_data_df.drop(columns =['CAGE_YR', 'CAGE_MO', 'RPT_DATE', 'ONSET_DATE','NUMDAYS', 'LAB_DATA', 
                                                   'V_ADMINBY', 'V_FUNDBY', 'OTHER_MEDS', 'CUR_ILL', 'PRIOR_VAX', 'SPLTTYPE', 
                                                   'FORM_VERS', 'TODAYS_DATE', 'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES'])
clean_vaers_data_df.head()

NameError: name 'vaers_data_df' is not defined

In [ ]:
# Verify Columns
clean_vaers_data_df.columns

In [ ]:
# Export Clean VAERSData to CSV for join in Postgres
clean_vaers_data_df.to_csv(r'../Resources/Data/clean_vaers_data.csv', index=True)

In [ ]:
# make sure L_THREAT is only 2 values
threat_check = vaers_data['L_THREAT'].unique()
threat_check

In [ ]:
# narrow down data to id and l threat
threat = vaers_data[['VAERS_ID','L_THREAT','AGE_YRS','SEX']]
threat

In [ ]:
threat.hist(column='AGE_YRS')

In [ ]:
# Set VAERS_ID as index
threat = threat.set_index(["VAERS_ID"])
threat.head()

In [ ]:
# narrow down ID's that only contain life threatening symptoms
threat_y = threat[threat['L_THREAT'] == 'Y']
threat_y

## VAERSVAX Cleanup

In [ ]:
# VAERSVAX Dataframe
vaers_vax = pd.read_csv(vax_csv, index_col=None, header=0, encoding='latin1', low_memory=False)
vaers_vax.head()

In [ ]:
# Set VAERS_ID as index
vaers_vax_df = vaers_vax.set_index(["VAERS_ID"])
vaers_vax_df

In [ ]:
# Check Columns
vaers_vax_df.columns

In [ ]:
# Drop  Columns 
clean_vaers_vax_df = vaers_vax_df.drop(columns =['VAX_LOT', 'VAX_DOSE_SERIES', 'VAX_ROUTE', 'VAX_SITE'])
clean_vaers_vax_df

## VAERS Symptoms Cleanup

In [ ]:
# Importing 2021VAERSSYMPTOMS.csv 
vaers_symptoms_df = pd.read_csv(symptoms_csv, index_col=None, header=0, encoding='latin1', low_memory=False)
vaers_symptoms_df.head()

In [ ]:
# converting NaN to 0 in symptoms df
vaers_symptoms_df = vaers_symptoms_df.fillna(0)
vaers_symptoms_df.head()

In [ ]:
# checking if it worked. yup it worked.
vaers_symptoms_df.isnull().sum()

In [ ]:
# Setting VAERS_ID as the index
vaers_symptoms_df = vaers_symptoms_df.set_index(["VAERS_ID"])
vaers_symptoms_df.head()

In [ ]:
# Dropping the SYMPTOMVERSION columns
new_vaers_symptoms_df = vaers_symptoms_df.drop(columns =['SYMPTOMVERSION1', 'SYMPTOMVERSION2', 'SYMPTOMVERSION3', 'SYMPTOMVERSION4', 'SYMPTOMVERSION5'])
new_vaers_symptoms_df.head()

In [ ]:
# Checking the data types
new_vaers_symptoms_df.dtypes

In [ ]:
new_vaers_symptoms_df.columns

In [ ]:
# Grouping SYMPTOM1 column by VAERS_ID and joingin the values
grouped_symptoms1 = new_vaers_symptoms_df.groupby('VAERS_ID')['SYMPTOM1'].apply(', '.join)
grouped_symptoms1.head()

In [ ]:
# Grouping SYMPTOM2 column by VAERS_ID and joingin the values
grouped_symptoms2_df = new_vaers_symptoms_df.astype(str).groupby('VAERS_ID')['SYMPTOM2'].apply(', '.join)
grouped_symptoms2_df.head()

In [ ]:
# Grouping SYMPTOM3 column by VAERS_ID and joingin the values
grouped_symptoms3 = new_vaers_symptoms_df.astype(str).groupby('VAERS_ID')['SYMPTOM3'].apply(', '.join)
grouped_symptoms3.head()

In [ ]:
# Grouping SYMPTOM4 column by VAERS_ID and joingin the values
grouped_symptoms4 = new_vaers_symptoms_df.astype(str).groupby('VAERS_ID')['SYMPTOM4'].apply(', '.join)
grouped_symptoms4.head()

In [ ]:
# Grouping SYMPTOM5 column by VAERS_ID and joingin the values
grouped_symptoms5 = new_vaers_symptoms_df.astype(str).groupby('VAERS_ID')['SYMPTOM5'].apply(', '.join)
grouped_symptoms5.head()

In [ ]:
# Merging Symptoms 1 and 2 by VAERS_ID
merge_1_2 = pd.merge(grouped_symptoms1, grouped_symptoms2_df, how="left", on=["VAERS_ID"])
merge_1_2.head()

In [ ]:
# Merging Symptoms 3 and 4 by VAERS_ID
merge_3_4 = pd.merge(grouped_symptoms3, grouped_symptoms4, how="left", on=["VAERS_ID"])
merge_3_4.head()

In [ ]:
# Merging Symptoms 1-4
merge_df = pd.merge(merge_1_2, merge_3_4, how="left", on=["VAERS_ID"])
merge_df.head()

In [ ]:
# Merging all symptom columns into one df
full_df_merge = pd.merge(merge_df, grouped_symptoms5, how="left", on=["VAERS_ID"])
full_df_merge.head(25)

In [ ]:
# using merge function by setting how='inner'
vax_symp_join = pd.merge(clean_vaers_vax_df, full_df_merge, 
                   on='VAERS_ID', 
                   how='inner')
vax_symp_join.head()

In [ ]:
# Verify Columns of vax_symp_join 
list(vax_symp_join.columns)

In [ ]:
# check to make sure only covid vaccines are in the list
covid_list = vax_symp_join['VAX_MANU'].unique()
covid_list

In [ ]:
vax_symp_join.head()

In [ ]:
covid_df = vax_symp_join[vax_symp_join["VAX_TYPE"] == 'COVID19']
covid_df

In [ ]:
# # check to make sure only covid vaccines are in the list
# covid_list = covid_df['VAX_MANU'].unique()
# covid_list

In [ ]:
# Exporting the VAX and Symptoms join
covid_df.to_csv(r'../Resources/Data/VAERS_Vax_Symptoms.csv', index=True)

## Machine Learning Cleanup

In [ ]:
#merge vaxsymp with l threat
merge_threat = pd.merge(covid_df, threat_y, on='VAERS_ID', how='right')
merge_threat

In [ ]:
ml_ready_df = merge_threat.drop(columns =['VAX_NAME','VAX_TYPE','L_THREAT'])
ml_ready_df

In [ ]:
# Export to CSV for Machine Learning
ml_ready_df.to_csv(r'../Resources/Data/MLdata.csv', index=True)